# 可迭代对象,迭代器和生成器

   - .[sentence第一版](#sentence第一版)
   - .[iterable and iterator](#iterable-and-iterator)
        * .[iterable](#iterable)
        * .[iterator](#iterator)
   - .[SentenceV2:典型的迭代器](#v2)
   - .[SentenceV3:生成器函数](#v3)
   - .[SentenceV4:惰性实现](#v4)
   - .[SentenceV5:生成器表达式](#v5)
   - .[生成器表达式](#expr)
   - .[等差数列生成器](#dcsl)
   - .[标准库中的生成器函数](#std)

## sentence第一版

In [3]:
import re 
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __getitem__(self, pos):
        return self.words[pos]
    
    def __len__(self):
        '''为了完善序列协议'''
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % self.text

In [9]:
s = Sentence('"The time has come," the Walrus said')
s

In [10]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [13]:
s[-1]

TypeError: 'Sentence' object is not subscriptable

In [12]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## iterable and iterator

### iterable
可以用iter()函数获取迭代器的对象

### iter函数 
先调用$__iter__$;如果没有则调用$__getitem__$,从0开始

### iterator

实现了无参数的$__next__$方法,返回序列中的下一个元素;如果没有则raise StopIteration


iterator要实现$__next__和__iter__$方法

## SentenceV2:典型的迭代器 <a id='v2'></a>

In [8]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __iter__(self):
        '''返回一个迭代器'''
        return SentenceIterator(self.words)
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration
        self.index += 1
        return word
    
    def __iter__(self):
        '''返回迭代器本身'''
        return self

不要给Sentence实现$__next__$方法,因为iterable要支持多种遍历,也就是支持从一个iterable实例中获取多个独立的iterator

## SentenceV3:生成器函数 <a id='v3'></a>

In [14]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __iter__(self):
        '''返回一个迭代器'''
        for word in self.words:
            yield word 
        

In [26]:
def gen_123():
    print('hello') # 创建实例时并不触发
    yield '1' # 隐式调用next
    print('world')
    yield '2'
    print('bye')
    yield '3'
    

g = gen_123()

In [27]:
# for机制的作用与g=iter(gen_123())一样,用于获取生成器对象
for a in g:
    print(a)

hello
1
world
2
bye
3


In [28]:
def def_1():
    print('hello')
    return 1

d = def_1()

hello


In [29]:
d

1

## SentenceV4:惰性实现 <a id='v4'></a>

lazy evaluation和eager evaluation相对,之前的版本急切得构建了文本中的单词列表,然后绑定到self.words属性上.列表使用的内存可能和文本一样多,而有些使用场景可能只需要查看前几个单词

In [30]:
class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __iter__(self):
        '''返回一个迭代器'''
        for match in RE_WORD.finditer(self.text):
            yield match.group() 

In [31]:
s = Sentence('"The time has come," the Walrus said')
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


## SentenceV5:生成器表达式 <a id='v5'></a>

用生成器表达式 (xx for xx in xxx)来代替生成器函数

In [32]:
class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __iter__(self):
        '''返回一个迭代器'''
        return (match.group() for match in RE_WORD.finditer(self.text))


In [33]:
s = Sentence('"The time has come," the Walrus said')
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


## 生成器表达式  <a id='expr'></a>  

如果函数只有一个参数时,不用多()  
如果一行写不完,最好使用生成器函数,提高readibility


## 等差数列生成器 <a id='dcsl'></a>

In [35]:
class ArithmeticProgression:
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step 
        self.end = end # end -> 无穷数列
    
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None 
        index = 0 
        while forever or result < self.end:
            yield result 
            index += 1
            result = self.begin + index * self.step # 不increment result是为了处理float时累计致错的风险
            

from fractions import Fraction
ap = ArithmeticProgression(1, Fraction(1, 3), 2)
for x in ap:
    print(x)        

1
4/3
5/3


In [36]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index
        

ap = aritprog_gen(1, Fraction(1, 3), 2)
next(ap)

Fraction(1, 1)

In [37]:
next(ap)

Fraction(4, 3)

In [38]:
next(ap)

Fraction(5, 3)

In [39]:
next(ap)

StopIteration: 

In [40]:
import itertools
gen = itertools.count(1, .5)  # 从不停止
next(gen)

1

In [41]:
next(gen)

1.5

In [42]:
next(gen)

2.0

In [44]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5)) # 用takewhile来停止
list(gen)

[1, 1.5, 2.0, 2.5]

## 标准库中的生成器函数 <a id='std'></a>